In [1]:
import numpy as np
import tensorflow as tf
from adult_modified import preprocess_adult_data
from sklearn import linear_model
import utils
import time
import multiprocessing as mp
import random
import matplotlib.pyplot as plt
import scipy
plt.ioff()
import sys
import os

In [2]:
def sample_perturbation(data_point, graph,  regularizer = 20, learning_rate = 3e-2, num_steps = 200):
    """
    Calculates ratio between perturbed loss and original loss

    parameters: 
        data_point: tuple of x, y
            x: tensor of shape (d, )
            y: one-hot encoded tensor of shape (2, )
        regularizer (float): regularizer constant for fair metric
        learning_rate (float): step size for gradient ascend
        num_steps (int): number of steps in gradient ascend

    return:
        float; ratio of entropy losses for perturbed and original sample
    """
    x, y = data_point
    x = tf.reshape(x, (1, -1))
    y = tf.reshape(y, (1, -1))
    x_start = x
    for _ in range(num_steps):
        with tf.GradientTape() as g:
            g.watch(x)
            prob = graph(x)
            perturb = utils.unprotected_direction(x-x_start, sensetive_directions)
            loss = utils.EntropyLoss(y, prob)  - regularizer  * tf.norm(perturb)**2

        gradient = g.gradient(loss, x)
        x = x + learning_rate * gradient

    return_loss = utils.EntropyLoss(y, graph(x)) / utils.EntropyLoss(y, graph(x_start))
    
    return return_loss.numpy()

In [3]:
seeds = np.load('../seeds.npy')
seed_data, seed_model = seeds[0,:]

In [8]:
lr = 0.001
iters = 20
dataset_orig_train, dataset_orig_test = preprocess_adult_data(seed = seed_data)

x_unprotected_train, x_protected_train = dataset_orig_train.features[:, :39], dataset_orig_train.features[:, 39:]
x_unprotected_test, x_protected_test = dataset_orig_test.features[:, :39], dataset_orig_test.features[:, 39:]
y_train, y_test = dataset_orig_train.labels.reshape((-1,)), dataset_orig_test.labels.reshape((-1,))



sensetive_directions = []
protected_regression = linear_model.LogisticRegression(fit_intercept = True)
protected_regression.fit(x_unprotected_test, x_protected_test[:, 0])
sensetive_directions.append(protected_regression.coef_.reshape((-1,)))
protected_regression.fit(x_unprotected_test, x_protected_test[:, 1])
sensetive_directions.append(protected_regression.coef_.reshape((-1,)))
sensetive_directions = np.array(sensetive_directions)

sensetive_directions = scipy.linalg.orth(sensetive_directions.T).T
for i, s in enumerate(sensetive_directions):
    while np.linalg.norm(s) != 1:
        s = s/ np.linalg.norm(s)
    sensetive_directions[i] = s

In [9]:
y_train, y_test = y_train.astype('int32'), y_test.astype('int32')
x_unprotected_train, x_unprotected_test = tf.cast(x_unprotected_train, dtype = tf.float32), tf.cast(x_unprotected_test, dtype = tf.float32)
y_train, y_test = tf.one_hot(y_train, 2), tf.one_hot(y_test, 2)
sensetive_directions = tf.cast(sensetive_directions, dtype = tf.float32)



graph = tf.keras.models.load_model(f'graphs/graph_{seed_data}_{seed_model}')  


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x14ce09ed0> and <tensorflow.python.keras.layers.core.Dense object at 0x14ce51310>).


In [10]:
graph

In [12]:
graph(x_unprotected_test)

<tf.Tensor: shape=(9045, 2), dtype=float32, numpy=
array([[0.4662716 , 0.5337285 ],
       [0.63381535, 0.3661847 ],
       [0.89712983, 0.1028702 ],
       ...,
       [0.56902593, 0.43097407],
       [0.97452027, 0.02547975],
       [0.9970921 , 0.00290783]], dtype=float32)>

In [13]:
data_point = x_unprotected_test[1], y_test[1] 

In [14]:
data_point

(<tf.Tensor: shape=(39,), dtype=float32, numpy=
 array([-0.49579805, -0.43515417,  0.26477247, -0.2192382 , -0.24259609,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0., 1.], dtype=float32)>)

In [15]:
sample_perturbation(data_point, graph)

19.128063